## Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings('ignore')
import logging
import time

# Configure the logger
logging.basicConfig(level=logging.WARNING)  # Adjust the log level as needed
# Set up a file handler to save logs to a file
log_file = 'data_cleaner.log'
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)  # Adjust the log level as needed
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)


### Raw Data

In [2]:
raw_data_path = Path("datasets/g20_raw_data.csv")  # Use forward slashes for the path

raw_data = pd.read_csv(raw_data_path)
COUNTRIES = raw_data["Country Name"].unique()
COUNTRIES = [country for country in COUNTRIES if pd.notna(country)]
print(COUNTRIES)

['Argentina', 'Australia', 'Brazil', 'Canada', 'China', 'France', 'Germany', 'India', 'Indonesia', 'Italy', 'United States', 'United Kingdom', 'Mexico', 'Japan', 'Korea, Rep.', 'Russian Federation', 'Saudi Arabia', 'South Africa', 'Turkiye', 'European Union']


In [3]:
# Create a mapping dictionary to rename the columns
column_mapping = {}
for column in raw_data.columns:
    if re.match(r'\d{4} \[YR\d{4}\]', column):
        # Extract the year part using regular expression
        year = re.search(r'\d{4}', column).group()
        column_mapping[column] = year

# Rename the columns
raw_data = raw_data.rename(columns=column_mapping)

In [4]:
raw_data.describe()

,Series Name,Series Code,Country Name,Country Code,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
count,1502,1500,1500,1500,1500,1500,1500,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
unique,77,75,20,20,716,732,787,813,817,823,...,1285,1295,1183,1281,1292,1263,1221,1168,832,588
top,Adjusted savings: gross savings (% of GNI),NY.ADJ.ICTR.GN.ZS,Argentina,ARG,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
freq,20,20,75,75,766,750,695,669,666,663,...,197,191,185,200,194,224,268,321,655,910


In [5]:
raw_data.head()

,Series Name,Series Code,Country Name,Country Code,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Adjusted savings: gross savings (% of GNI),NY.ADJ.ICTR.GN.ZS,Argentina,ARG,..,..,..,34.03839404,32.76262784,30.37099055,...,15.31971022,16.10743905,14.56261556,14.96386557,13.41810079,11.82578069,14.87426431,19.11757389,..,..
1,Adjusted savings: gross savings (% of GNI),NY.ADJ.ICTR.GN.ZS,Australia,AUS,..,..,..,..,..,..,...,24.58845708,24.30342107,23.17352013,21.21821891,22.41241032,22.4413726,23.27337379,24.57498917,..,..
2,Adjusted savings: gross savings (% of GNI),NY.ADJ.ICTR.GN.ZS,Brazil,BRA,..,..,21.83711367,19.52124571,20.11643795,19.95870114,...,18.40746571,16.44076791,14.80798801,13.71267295,13.83644283,13.07087306,12.88752293,15.33326854,..,..
3,Adjusted savings: gross savings (% of GNI),NY.ADJ.ICTR.GN.ZS,Canada,CAN,23.54913717,24.48415402,21.43182218,22.25910036,21.05833391,20.83639622,...,21.93134446,22.51225199,19.87382492,19.38325078,20.10090864,19.83495477,20.27153848,18.90878582,..,..
4,Adjusted savings: gross savings (% of GNI),NY.ADJ.ICTR.GN.ZS,China,CHN,..,..,..,..,..,..,...,47.76683415,47.55544345,45.63534427,44.60182309,44.9639649,44.68200336,43.896015,44.85112622,..,..


In [6]:
# Extract columns for years from raw_data
years_columns = raw_data.columns[4:]

In [7]:
# Get Availble columns
print(raw_data.columns)

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022'],
      dtype='object')


In [8]:
# Types of Data available
data_available = raw_data['Series Name'].unique()
DATA_NAME = data_available
# Convert all elements to strings
data_available = [str(value) for value in data_available]

print(data_available)

['Adjusted savings: gross savings (% of GNI)', 'Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)', 'Bank nonperforming loans to total gross loans (%)', 'Claims on other sectors of the domestic economy (% of GDP)', 'Claims on other sectors of the domestic economy (annual growth as % of broad money)', 'Domestic credit provided by financial sector (% of GDP)', 'Domestic credit to private sector (% of GDP)', 'Domestic credit to private sector by banks (% of GDP)', 'Domestic general government health expenditure (% of current health expenditure)', 'Domestic general government health expenditure (% of GDP)', 'Domestic general government health expenditure (% of general government expenditure)', 'Domestic general government health expenditure per capita (current US$)', 'Domestic general government health expenditure per capita, PPP (current international $)', 'Domestic private health expenditure (% of current health expenditure)', 'Domestic private health expenditure

## 2d dataframe

In [9]:
# Create a new DataFrame with 'country' as the first column and unique values from 'Series Name' as columns
df = pd.DataFrame(columns=['Countries'] + data_available)

# Drop the specified columns only if they exist in the DataFrame
columns_to_remove = ['nan', 'Last Updated: 10/26/2023']
columns_to_remove = [col for col in columns_to_remove if col in df.columns]

df = df.drop(columns=columns_to_remove)

# Add G20 Countries
df["Countries"] = COUNTRIES

display(df)

,Countries,Adjusted savings: gross savings (% of GNI),"Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)",Bank nonperforming loans to total gross loans (%),Claims on other sectors of the domestic economy (% of GDP),Claims on other sectors of the domestic economy (annual growth as % of broad money),Domestic credit provided by financial sector (% of GDP),Domestic credit to private sector (% of GDP),Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of current health expenditure),...,"School enrollment, secondary (% gross)","School enrollment, secondary (gross), gender parity index (GPI)","School enrollment, secondary, female (% gross)","School enrollment, secondary, male (% gross)","School enrollment, tertiary (% gross)","School enrollment, tertiary (gross), gender parity index (GPI)","School enrollment, tertiary, female (% gross)","School enrollment, tertiary, male (% gross)","Stocks traded, turnover ratio of domestic shares (%)",Data from database: World Development Indicators
0,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Indonesia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Define the range of years
YEARS = range(1973, 2023)

# Create a MultiIndex for the index
index = pd.MultiIndex.from_product([COUNTRIES, YEARS], names=['country', 'year'])

# Create an empty DataFrame with the MultiIndex as the index
g20_data = pd.DataFrame(index=index)

# Add Columns with data
for col in data_available:
    g20_data[col] = None

In [11]:
# Assuming you have a DataFrame with a MultiIndex, such as 'g20_data'
# You can access the MultiIndex and its structure like this:

multi_index = g20_data.index

# To get the levels of the MultiIndex
levels = multi_index.levels

# To get the names of the levels
level_names = multi_index.names

# To get the number of levels in the MultiIndex
num_levels = multi_index.nlevels

# Print the information
print("MultiIndex Levels:", levels)
print("MultiIndex Level Names:", level_names)
print("Number of MultiIndex Levels:", num_levels)

MultiIndex Levels: [['Argentina', 'Australia', 'Brazil', 'Canada', 'China', 'European Union', 'France', 'Germany', 'India', 'Indonesia', 'Italy', 'Japan', 'Korea, Rep.', 'Mexico', 'Russian Federation', 'Saudi Arabia', 'South Africa', 'Turkiye', 'United Kingdom', 'United States'], [1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]]
MultiIndex Level Names: ['country', 'year']
Number of MultiIndex Levels: 2


In [12]:
g20_data.head()

Adjusted savings: gross savings (% of GNI)  \
country   year                                              
Argentina 1973                                       None   
          1974                                       None   
          1975                                       None   
          1976                                       None   
          1977                                       None   

               Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)  \
country   year                                                                              
Argentina 1973                                               None                           
          1974                                               None                           
          1975                                               None                           
          1976                                               None                           
          1977                                               None                           

               Bank nonperforming loans to total gross loans (%)  \
country   year                                                     
Argentina 1973                                              None   
          1974                                              None   
          1975                                              None   
          1976                                              None   
          1977                                              None   

               Claims on other sectors of the domestic economy (% of GDP)  \
country   year                                                              
Argentina 1973                                               None           
          1974                                               None           
          1975                                               None           
          1976                                               None           
          1977                                               None           

               Claims on other sectors of the domestic economy (annual growth as % of broad money)  \
country   year                                                                                       
Argentina 1973                                               None                                    
          1974                                               None                                    
          1975                                               None                                    
          1976                                               None                                    
          1977                                               None                                    

               Domestic credit provided by financial sector (% of GDP)  \
country   year                                                           
Argentina 1973                                               None        
          1974                                               None        
          1975                                               None        
          1976                                               None        
          1977                                               None        

               Domestic credit to private sector (% of GDP)  \
country   year                                                
Argentina 1973                                         None   
          1974                                         None   
          1975                                         None   
          1976                                         None   
          1977                                         None   

               Domestic credit to private sector by banks (% of GDP)  \
country   year                                                         
Argentina 1973                                               None      
          1974            

### 3d Dataframe

In [13]:
# Measure the start time
start_time = time.time()

# Create a single tqdm loop for the outermost iteration
progress_bar = tqdm(total=len(COUNTRIES) * len(YEARS) * len(DATA_NAME), bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]", position=0)

# Iterate over countries, years, and data names
for country in COUNTRIES:
    for year in YEARS:
        for index in DATA_NAME:
            try:
                description = f"Modifying {country} on {year}"
                progress_bar.set_description(description)

                # Construct the column name corresponding to the year
                year_column = str(year)

                # Filter the data based on the current country, year, and data name
                filtered_data = raw_data[(raw_data['Series Name'] == index) & (raw_data['Country Name'] == country)][year_column]

                # Check if any data is found for the current combination
                if not filtered_data.empty:
                    # Assuming you want to update the 'g20_data' DataFrame with the filtered data
                    # You need to provide the column in 'g20_data' where you want to store the filtered data
                    # Here, we assume 'index' is the column name in 'g20_data'
                    g20_data.loc[(country, year), index] = filtered_data.values[0]
            except Exception as e:
                logger.error(f"Error for {country} on {year} for {index}: {str(e)}")

            progress_bar.update(1)

# Remove useles columns
columns_to_remove = ['nan','Data from database: World Development Indicators', 'Last Updated: 10/26/2023']
g20_data = g20_data.drop(columns=columns_to_remove)

# Close the tqdm progress bar
progress_bar.close()

# Measure the end time
end_time = time.time()

# Calculate and log the elapsed time
elapsed_time = end_time - start_time
logger.info(f"Total elapsed time to make multidimesional Dataframe: {elapsed_time:.2f} seconds")
print(f"Process Completed in {elapsed_time:.2f} seconds")

  0%|          | 0/78000 [00:00<?]

Process Completed in 121.90 seconds


In [14]:
g20_data.head()

Adjusted savings: gross savings (% of GNI)  \
country   year                                              
Argentina 1973                                         ..   
          1974                                         ..   
          1975                                         ..   
          1976                                34.03839404   
          1977                                32.76262784   

               Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)  \
country   year                                                                              
Argentina 1973                                                 ..                           
          1974                                                 ..                           
          1975                                                 ..                           
          1976                                                 ..                           
          1977                                                 ..                           

               Bank nonperforming loans to total gross loans (%)  \
country   year                                                     
Argentina 1973                                                ..   
          1974                                                ..   
          1975                                                ..   
          1976                                                ..   
          1977                                                ..   

               Claims on other sectors of the domestic economy (% of GDP)  \
country   year                                                              
Argentina 1973                                                 ..           
          1974                                                 ..           
          1975                                                 ..           
          1976                                                 ..           
          1977                                                 ..           

               Claims on other sectors of the domestic economy (annual growth as % of broad money)  \
country   year                                                                                       
Argentina 1973                                                 ..                                    
          1974                                                 ..                                    
          1975                                                 ..                                    
          1976                                                 ..                                    
          1977                                                 ..                                    

               Domestic credit provided by financial sector (% of GDP)  \
country   year                                                           
Argentina 1973                                   27.4901879012225        
          1974                                    30.371312312636        
          1975                                   28.7594771456156        
          1976                                   20.3441509421658        
          1977                                   26.4137652814314        

               Domestic credit to private sector (% of GDP)  \
country   year                                                
Argentina 1973                              16.705575724589   
          1974                             19.1738082781793   
          1975                             16.4413498377265   
          1976                             13.5627672947772   
          1977                             18.3829098551528   

               Domestic credit to private sector by banks (% of GDP)  \
country   year                                                         
Argentina 1973                                   15.6482608053113      
          1974            

In [15]:
g20_data.tail()

Adjusted savings: gross savings (% of GNI)  \
country        year                                              
European Union 2018                           25.2294177448061   
               2019                           25.5716180198675   
               2020                           24.6958165282482   
               2021                                         ..   
               2022                                         ..   

                    Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)  \
country        year                                                                              
European Union 2018                                   23.0030025157958                           
               2019                                   23.2552120614057                           
               2020                                   23.2594038346861                           
               2021                                                 ..                           
               2022                                                 ..                           

                    Bank nonperforming loans to total gross loans (%)  \
country        year                                                     
European Union 2018                                                ..   
               2019                                                ..   
               2020                                                ..   
               2021                                                ..   
               2022                                                ..   

                    Claims on other sectors of the domestic economy (% of GDP)  \
country        year                                                              
European Union 2018                                                 ..           
               2019                                                 ..           
               2020                                                 ..           
               2021                                                 ..           
               2022                                                 ..           

                    Claims on other sectors of the domestic economy (annual growth as % of broad money)  \
country        year                                                                                       
European Union 2018                                                 ..                                    
               2019                                                 ..                                    
               2020                                                 ..                                    
               2021                                                 ..                                    
               2022                                                 ..                                    

                    Domestic credit provided by financial sector (% of GDP)  \
country        year                                                           
European Union 2018                                                 ..        
               2019                                                 ..        
               2020                                                 ..        
               2021                                                 ..        
               2022                                                 ..        

                    Domestic credit to private sector (% of GDP)  \
country        year                                                
European Union 2018                             86.3004747661652   
               2019                             85.3848366373926   
               2020                             93.4769835714633   
               2021                             89.8812259606375   
               2022                             86.2434768450053   

  

In [16]:
# Assuming you have a DataFrame with a MultiIndex, such as 'g20_data'
# You can access the MultiIndex and its structure like this:

multi_index = g20_data.index

# To get the levels of the MultiIndex
levels = multi_index.levels

# To get the names of the levels
level_names = multi_index.names

# To get the number of levels in the MultiIndex
num_levels = multi_index.nlevels

# Print the information
print("MultiIndex Levels:", levels)
print("MultiIndex Level Names:", level_names)
print("Number of MultiIndex Levels:", num_levels)

MultiIndex Levels: [['Argentina', 'Australia', 'Brazil', 'Canada', 'China', 'European Union', 'France', 'Germany', 'India', 'Indonesia', 'Italy', 'Japan', 'Korea, Rep.', 'Mexico', 'Russian Federation', 'Saudi Arabia', 'South Africa', 'Turkiye', 'United Kingdom', 'United States'], [1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]]
MultiIndex Level Names: ['country', 'year']
Number of MultiIndex Levels: 2


In [17]:
g20_data.columns

Index(['Adjusted savings: gross savings (% of GNI)',
       'Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)',
       'Bank nonperforming loans to total gross loans (%)',
       'Claims on other sectors of the domestic economy (% of GDP)',
       'Claims on other sectors of the domestic economy (annual growth as % of broad money)',
       'Domestic credit provided by financial sector (% of GDP)',
       'Domestic credit to private sector (% of GDP)',
       'Domestic credit to private sector by banks (% of GDP)',
       'Domestic general government health expenditure (% of current health expenditure)',
       'Domestic general government health expenditure (% of GDP)',
       'Domestic general government health expenditure (% of general government expenditure)',
       'Domestic general government health expenditure per capita (current US$)',
       'Domestic general government health expenditure per capita, PPP (current international $)',
       'Domestic pr

In [18]:
# Access data for 'Argentina' in the year '1973' for all columns
data = g20_data.loc[('Argentina', 1973), :]


In [19]:
display(data)

Adjusted savings: gross savings (% of GNI)                                                            ..
Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)                            ..
Bank nonperforming loans to total gross loans (%)                                                     ..
Claims on other sectors of the domestic economy (% of GDP)                                            ..
Claims on other sectors of the domestic economy (annual growth as % of broad money)                   ..
                                                                                             ...        
School enrollment, tertiary (% gross)                                                   19.7516307830811
School enrollment, tertiary (gross), gender parity index (GPI)                         0.847370028495789
School enrollment, tertiary, female (% gross)                                           18.1059703826904
School enrollment, tertiary, male (% gross)            